In [ ]:
from __future__ import unicode_literals, print_function, division
import string
import gc
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import nltk
nltk.download('punkt')
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
import pickle
from scipy.stats import skew
import cv2
import math
from google.colab import files
from google.colab.patches import cv2_imshow
import numpy as np
from string import printable
from re import X
from IPython.core import formatters
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
import xml.etree.ElementTree as ET
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from io import open
import unicodedata

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
a = np.array([[[1,1],[2,2]], [[3,3],[4,4]]])
print(np.array([[[1,1],[2,2]], [[3,3],[4,4]]]).shape)
a_reshaped = a.reshape(a.shape[0], -1)
print(a)
print(a_reshaped)
np.savetxt('test1.txt', a_reshaped)
b = np.loadtxt('test1.txt')
print('')
print(b)
b = b.reshape(len(b), len(a[0]), a.shape[2])
print(b)
a == b
# array([ True,  True,  True,  True], dtype=bool)

(2, 2, 2)
[[[1 1]
  [2 2]]

 [[3 3]
  [4 4]]]
[[1 1 2 2]
 [3 3 4 4]]

[[1. 1. 2. 2.]
 [3. 3. 4. 4.]]
[[[1. 1.]
  [2. 2.]]

 [[3. 3.]
  [4. 4.]]]


array([[[ True,  True],
        [ True,  True]],

       [[ True,  True],
        [ True,  True]]])

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?']+", r" ", s)
    return s

MAX_LENGTH =  300

In [ ]:
videos = ['/content/rachel1.mp4', '/content/1-Ben-Introduction-SD.mov']
xml = ['/content/xml_extract_Rachel_2012-01-27_sc51_9.xml', 'xml_extract_1-Ben-Introduction_17.xml']

sequences =  []
translatedSequences =  []
AllSigns = []
AllFrames = []
AllFramesUncompressed = []
Labels = []

for video, xml in zip(videos, xml):
  capture = cv2.VideoCapture(video)
  count = 0
  frames = []
  frameNr = 0
  
  while (True):
      success, frame = capture.read()
      if success:
          frame = cv2.resize(frame, (320,240), interpolation = cv2.INTER_AREA)
          frames.append(frame)
      else:
          break
      count = count+1
  capture.release()

  print(len(frames))

  tree = ET.parse(xml)
  
  # getting the parent tag of
  # the xml document
  root = tree.getroot()
  
  # printing the root (parent) tag
  # of the xml document, along with
  # its memory location
  print(root)
  
  # printing the attributes of the
  # first tag from the parent
  print(root[0].attrib)
  
  # printing the text contained within
  # first subtag of the 5th tag from
  # the parent
  print(root[0][0][3][0][0][0][2])

  utterances= tree.findall('.//UTTERANCE')
  signs = utterances[1][3].findall('.//SIGN')
  UtteranceSigns = utterances[1][3].findall('.//SIGN')

  for sign  in tree.findall('.//SIGN'):
    # check if exists in unique_list or not
    if sign[0].text not in AllSigns:
      AllSigns.append(sign[0].text)

  label_map1 = {num:label for num, label in enumerate(AllSigns)}
  label_map2 = {label:num for num, label in enumerate(AllSigns)}

  
  for utterance in utterances:
      UtteranceSigns = utterance[3].findall('.//SIGN')
      UtteranceSignsSorted = sorted(UtteranceSigns, key=lambda x: x[13].get('START_FRAME'))
      utteranceFrameSigns = []
      signIDs = []
      FrameSequence = []
      FrameSequenceUncompressed = []
      SignSequence = ""
      a=0
      for sign in UtteranceSignsSorted:
        if sign.get('ID') not in signIDs:
          signIDs.append(sign.get('ID'))
          for frame_num in range(int(int(sign[13].get('START_FRAME'))/1000),int(int(sign[13].get('END_FRAME'))/1000)):
            if (frame_num - (int(int(sign[13].get('START_FRAME'))/1000)) ) % 3 == 0:
              FrameSequence.append(frames[frame_num-1].flatten())
              FrameSequenceUncompressed.append(frames[frame_num-1])
              utteranceFrameSigns.append(label_map2[sign[0].text])
      Labels.append(utteranceFrameSigns)
      translatedSequences.append(normalizeString(utterance[0].text))
      sequences.append(normalizeString(SignSequence))
      AllFrames.append(FrameSequence)
      AllFramesUncompressed.append(FrameSequenceUncompressed)

6996
<Element 'SIGNSTREAM_DAI' at 0x7f4e17caa650>
{}
<Element 'UTTERANCES' at 0x7f4e17caafb0>
4409
<Element 'SIGNSTREAM_DAI' at 0x7f4e17cc4890>
{}
<Element 'UTTERANCES' at 0x7f4e17cd50b0>


In [ ]:
skinMask360Sequences = []
for utterance in AllFramesUncompressed:
  skinMask360Utterance = [] 
  for frame in utterance:

    img = frame.copy()
    img_bw = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # to be considered 'skin'
    hsvim = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower = np.array([0, 48, 80], dtype="uint8")
    upper = np.array([30, 255, 255], dtype="uint8")
    skinMask= cv2.inRange(hsvim, lower, upper)

    # blur the mask to help remove noise
    skinMask = cv2.blur(skinMask, (15, 15))

    # get threshold image
    ret, thresh = cv2.threshold(skinMask, 100, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    distancesPerBlob = []

    # find contours in the binary image
    # Convert source image to unsigned 8 bit integer Numpy array
    # help(cv2.cvtColor)
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    cntsSorted = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)

    count= 0


    for c in cntsSorted:

      if(count == 3):
        break;

      count = count+1;

      # Step #3
      out = img.copy()

      # Step #4
      ref = np.zeros_like(img_bw)
      cv2.drawContours(ref, c, -1, 255, 5)


      width = img.shape[1]
      height = img.shape[0]

      # Define total number of angles we want
      N = 360

      # Step #5
      M = cv2.moments(c)

      if(M['m00'] == 0):
        count = count-1;
        continue

      centroid_x = int(M['m10']/M['m00'])
      centroid_y = int(M['m01']/M['m00'])

      # Step #6
      for i in range(N):
        # Step #6a
        tmp = np.zeros_like(img_bw)

        # Step #6b
        theta = i*(360/N)
        theta *= np.pi/180.0

        # Step #6c
        cv2.line(tmp, (centroid_x, centroid_y),
        (int(centroid_x+np.cos(theta)*width),
        int(centroid_y-np.sin(theta)*height)), 255, 5)

        # Step #6d
        (row,col) = np.nonzero(np.logical_and(tmp, ref))

        # Step #6e
        try:
          cv2.line(out, (centroid_x, centroid_y), (col[0],row[0]), 0, 1)
          distancesPerBlob.append(math.hypot(col[0] - centroid_x, row[0] - centroid_y))
        except:
          distancesPerBlob.append(0)
      
    while len(distancesPerBlob) < 1080:
        distancesPerBlob.append(0)
    skinMask360Utterance.append(distancesPerBlob)
  skinMask360Sequences.append(skinMask360Utterance)

In [ ]:
padded_inputs2 = tf.keras.preprocessing.sequence.pad_sequences(
    skinMask360Sequences, padding="post", value=-1
)

In [ ]:
padded_inputs2.shape

(56, 93, 1080)

In [ ]:
a2 = np.array(padded_inputs2)
print(np.array(padded_inputs2).shape)
a2_reshaped = a2.reshape(a2.shape[0], -1)
print(a2)
print(a2_reshaped)
np.savetxt('360Arrays.txt', a2_reshaped)
b2 = np.loadtxt('360Arrays.txt')
print('')
print(b2.shape)
b2 = b2.reshape(len(b2), len(a2[0]), a2.shape[2])
print(b2.shape)
a2 == b2
# array([ True,  True,  True,  True], dtype=bool)

(56, 93, 1080)
[[[10 10 10 ...  0  0  0]
  [11 11 11 ...  0  0  0]
  [10 10 10 ...  0  0  0]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 [[22 22 23 ...  0  0  0]
  [24 24 24 ...  0  0  0]
  [22 22 22 ...  0  0  0]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 [[17 17 18 ... 14 14 14]
  [16 16 18 ...  8  8  8]
  [17 17 18 ... 15 15 15]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 ...

 [[20 20 21 ...  0  0  0]
  [24 24 24 ...  0  0  0]
  [20 20 21 ...  0  0  0]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 [[21 21 21 ... 16 16 16]
  [21 21 21 ...  5  5  5]
  [18 18 19 ...  0  0  0]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]]

 [[18 18 18 ...  0  0  0]
  [22 22 23 ...  0  0  0]
  [10 10 10 ...  0  0  0]
  ...
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -1]
  [-1 -1 -1 ... -1 -1 -

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  Tr

In [ ]:
b2.shape

(56, 93, 1080)